In [1]:

from typing import Annotated

from langchain_core.tools import tool
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnableConfig

from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, tools_condition


@tool
def divide(a: float, b: float) -> int:
    """Return a / b."""
    return a / b

llm = ChatOllama(model="llama3.1:8b")

tools = [divide]
memory = MemorySaver()

class State(TypedDict):
    messages: Annotated[list, add_messages]

config = RunnableConfig(
    configurable={"thread_id": "1"},  # 스레드 ID 설정
)

graph_builder = StateGraph(State)
graph_builder.add_node("tools", ToolNode(tools))
graph_builder.add_node("chatbot", lambda state: {"messages":llm.bind_tools(tools).invoke(state['messages'])})
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges(
    "chatbot", tools_condition
)
graph_builder.set_entry_point("chatbot")

graph = graph_builder.compile(checkpointer=memory)
# 첫 질문
question = (
    "What's 329993 divided by 13662?"
)

for event in graph.stream({"messages": [("user", question)]}, config=config):
    for value in event.values():
        print(value)

{'messages': AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-02-05T07:07:04.175525Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7176196375, 'load_duration': 941731500, 'prompt_eval_count': 173, 'prompt_eval_duration': 4189000000, 'eval_count': 24, 'eval_duration': 2043000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-3726a73e-c8e3-4fa4-9179-55ce3e7b8ec0-0', tool_calls=[{'name': 'divide', 'args': {'a': 329993, 'b': 13662}, 'id': '966295b5-1441-4e58-8610-38ca61282e21', 'type': 'tool_call'}], usage_metadata={'input_tokens': 173, 'output_tokens': 24, 'total_tokens': 197})}
{'messages': [ToolMessage(content='24.15407700190309', name='divide', id='4e061dd3-1064-46ef-9d72-6ae77a08492a', tool_call_id='966295b5-1441-4e58-8610-38ca61282e21')]}
{'messages': AIMessage(content='The result of dividing 329993 by 13662 is approximately 24.15.', additional_kwargs={}, response_metadata=

In [6]:
snapshot = graph.get_state(config)
print(snapshot.values)
print("------"*8)
print(snapshot.next)
print("------"*8)
print(snapshot.config)
print("------"*8)
print(snapshot.metadata)

{'messages': [HumanMessage(content="What's 329993 divided by 13662?", additional_kwargs={}, response_metadata={}, id='67c0bcd7-e15c-4196-b29d-55f983a75724'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-02-05T07:07:04.175525Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7176196375, 'load_duration': 941731500, 'prompt_eval_count': 173, 'prompt_eval_duration': 4189000000, 'eval_count': 24, 'eval_duration': 2043000000, 'message': {'role': 'assistant', 'content': '', 'images': None, 'tool_calls': None}}, id='run-3726a73e-c8e3-4fa4-9179-55ce3e7b8ec0-0', tool_calls=[{'name': 'divide', 'args': {'a': 329993, 'b': 13662}, 'id': '966295b5-1441-4e58-8610-38ca61282e21', 'type': 'tool_call'}], usage_metadata={'input_tokens': 173, 'output_tokens': 24, 'total_tokens': 197}), ToolMessage(content='24.15407700190309', name='divide', id='4e061dd3-1064-46ef-9d72-6ae77a08492a', tool_call_id='966295b5-1441-4e58-8610-38ca61282e21'), AI